In [1]:
import numpy as np
from keras.utils import np_utils
np.random.seed(10)
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense

Using TensorFlow backend.


In [2]:
#建立訓練資料和測試資料，包括訓練特徵集、訓練標籤和測試特徵集、測試標籤	
(train_feature, train_label),\
(test_feature, test_label) = mnist.load_data() 

In [3]:
# 將 Features 特徵值換為 60000*28*28*1 的 4 維矩陣
train_feature_vector =train_feature.reshape(len(train_feature), 28,28,1).astype('float32')
test_feature_vector = test_feature.reshape(len( test_feature), 28,28,1).astype('float32')

In [4]:
# Features 特徵值標準化
train_feature_normalize = train_feature_vector/255
test_feature_normalize = test_feature_vector/255

In [5]:
# label 轉換為 One-Hot Encoding 編碼
train_label_onehot = np_utils.to_categorical(train_label)
test_label_onehot = np_utils.to_categorical(test_label)

In [6]:
# 建立模型
model = Sequential()

W0829 16:48:09.398148 36708 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [7]:
# 建立卷積層 1
model.add(Conv2D(filters=10, 
                 kernel_size=(5,5),
                 padding='same',
                 input_shape=(28,28,1), 
                 activation='relu'))

W0829 16:48:12.183421 36708 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 16:48:12.186422 36708 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [8]:
# 建立池化層 1
model.add(MaxPooling2D(pool_size=(2, 2))) #(10,14,14)

W0829 16:48:13.534589 36708 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [9]:
# 建立卷積層 2
model.add(Conv2D(filters=20, 
                 kernel_size=(5,5),  
                 padding='same',
                 activation='relu'))

In [10]:
# 建立池化層 2
model.add(MaxPooling2D(pool_size=(2, 2))) #(20,7,7)

In [11]:
# Dropout層防止過度擬合，斷開比例:0.2
model.add(Dropout(0.2))

W0829 16:48:18.629522 36708 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0829 16:48:18.637524 36708 deprecation.py:506] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
# 建立平坦層：20*7*7=980 個神經元
model.add(Flatten()) 

In [13]:
# 建立隱藏層
model.add(Dense(units=256, activation='relu'))

In [14]:
# 建立輸出層
model.add(Dense(units=10,activation='softmax'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 10)        260       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 10)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 20)        5020      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 20)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 20)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 980)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               251136    
__________

In [16]:
# 這些訓練會累積，準確率會愈來愈高
try:
    model.load_weights("Mnist_cnn_model.weight")
    print("載入模型參數成功，繼續訓練模型!")
except :    
    print("載入模型失敗，開始訓練一個新模型!")

載入模型失敗，開始訓練一個新模型!


In [17]:
# 定義訓練方式
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

W0829 16:48:30.262273 36708 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0829 16:48:30.284268 36708 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [18]:
# 以(train_feature_normalize,train_label_onehot)資料訓練，
# 訓練資料保留 20% 作驗證,訓練 10 次、每批次讀取 200 筆資料，顯示完整訓練過程
train_history =model.fit(x=train_feature_normalize,
                         y=train_label_onehot,validation_split=0.2, 
                         epochs=10, batch_size=200,verbose=1)

W0829 16:48:32.961814 36708 deprecation.py:323] From D:\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 20s 420us/step - loss: 0.3325 - acc: 0.9030 - val_loss: 0.0886 - val_acc: 0.9733
Epoch 2/10
48000/48000 [==============================] - 20s 410us/step - loss: 0.0869 - acc: 0.9730 - val_loss: 0.0647 - val_acc: 0.9804
Epoch 3/10
48000/48000 [==============================] - 20s 410us/step - loss: 0.0633 - acc: 0.9801 - val_loss: 0.0509 - val_acc: 0.9845
Epoch 4/10
48000/48000 [==============================] - 20s 409us/step - loss: 0.0514 - acc: 0.9838 - val_loss: 0.0475 - val_acc: 0.9859
Epoch 5/10
48000/48000 [==============================] - 20s 410us/step - loss: 0.0408 - acc: 0.9871 - val_loss: 0.0421 - val_acc: 0.9864
Epoch 6/10
48000/48000 [==============================] - 20s 411us/step - loss: 0.0348 - acc: 0.9894 - val_loss: 0.0381 - val_acc: 0.9878
Epoch 7/10
48000/48000 [==============================] - 20s 408us/step - loss: 0.0306 - acc: 0.9899 - val_loss: 0.

In [19]:
#評估準確率
scores = model.evaluate(test_feature_normalize, test_label_onehot)
print('\n準確率=',scores[1])

10000/10000 [==============================] - 1s 131us/step

準確率= 0.9907


In [20]:
# 儲存模型
model.save('Mnist_cnn_model.h5')
print("\nMnist_cnn_model.h5 模型儲存完畢!")
model.save_weights("Mnist_cnn_model.weight")
print("Mnist_cnn_model.weight 模型參數儲存完畢!")


Mnist_cnn_model.h5 模型儲存完畢!
Mnist_cnn_model.weight 模型參數儲存完畢!


In [21]:
del model     